In [1]:
#library import
import math
import wave
import scipy
import librosa
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import os
import glob
import librosa.display
import soundfile as sf
from pydub import AudioSegment
import matplotlib.pyplot as plt
from IPython.display import Audio
from pydub.utils import db_to_float
from pydub.silence import split_on_silence
from pydub.silence import detect_silence

c:\Users\maxme\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
files_dir ="wav_files"

In [3]:
label_df=pd.read_csv("recordings-diagnosis.csv")

In [7]:
label_df["professional-diagnosis"].value_counts()

True     39257
False    22780
Name: professional-diagnosis, dtype: int64

In [ ]:
df = pd.DataFrame(columns=['id','class','duration','maximum_pitch','minmum_pitch','mean_pitch',
'total_energy','energy_freq','power','minimum_intensity','maximum_intensity','mean_intensity','speaking_rate',
'silences','voice_breaks','total_duration_of_pauses','number_of_pauses',
'number_of_voice_breaks','maximum_duration_of_pauses','avg_duration_of_pauses',
'maximum_falling','maximum_rising','jitter','shimmer','jitterrap','number_of_rising',
'number_of_falling','avg_rise','avg_fall','zcr','chroma_stft','mfcc','root_mean_square_value','melspectogram'
,'spectral_bandwidth','spectral_centroid'])

In [ ]:
len(df.columns)

In [ ]:
os.listdir(files_dir)[0]

In [ ]:
#Loading data

save = []
for index,file in enumerate(os.listdir(files_dir)[:5000]):
    save.append(file[:-4])
    save.append(label_df["professional-diagnosis"].loc[label_df["file_name"]==file].values[0])
    data, sampling_rate  = librosa.core.load(os.path.join(files_dir,file))
    Audio(data = data,rate = sampling_rate)
    librosa.display.waveshow(data, sampling_rate)
    # Duration : Total duration of the audio
    total_samples = data.size
    total_duration = int(total_samples/sampling_rate)
    save.append(total_duration)
    # Fourier Transform from data 

    fourier_transform = np.fft.rfft(data)

    # Frequencies

    frequencies = np.abs(fourier_transform)
    
    # Maximum Pitch

    max_pitch = np.argmax(frequencies)
    save.append(max_pitch)

    arr = [117]*len(frequencies)
    sum_freq = 0
    for i in range(0,len(frequencies)):
        if int(frequencies[i])>0 and int(frequencies[i])<int(max_pitch):
            arr[i] = int(frequencies[i])
            sum_freq=sum_freq + i
    # Minmum Pitch
    min_pitch = arr.index(min(arr))
    save.append(min_pitch)
    # Mean pitch
    mean_pitch=sum_freq/len(frequencies)
    save.append(mean_pitch)
    #total energy
    energy= np.sum(data**2)
    save.append(energy)
    energy_freq = np.sum(frequencies**2)/total_samples
    save.append(energy_freq)
    #power
    power = energy / total_duration
    save.append(power)
    #minimum intensity
    plt.figure(1, figsize=(9,6))
    plt.subplot(211)
    #IntensityMin : Minimum Intensity of the audio signal
    Pxx, freqs, bins, im = plt.specgram(data, Fs = sampling_rate, NFFT=1024, cmap=plt.get_cmap('autumn_r'))
    col_bar=plt.colorbar(im)
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    col_bar.set_label('Intensity dB')
    plt.show()
    plt.close()
    min_intensity = col_bar.vmin
    save.append(min_intensity)
    #maximum intensity
    max_intensity = col_bar.vmax
    save.append(max_intensity)
    #mean intensity
    mean_intensity = (min_intensity + max_intensity) / 2
    save.append(mean_intensity)

    sound = AudioSegment.from_wav(os.path.join(files_dir,file))
    len(sound)
    chunks = split_on_silence(sound, 
    # must be silent for at least half a second
    min_silence_len=20,

    # consider it silent if quieter than -16 dBFS
        silence_thresh=-(round(abs(sound.dBFS))+100)
    )



    # 1000 means 1 sec then 60 second means
    sixty_seconds = 60 * 1000

    second_1_minute = sound[sixty_seconds:120000]
    hunks = split_on_silence(second_1_minute, 
        # must be silent for at least half a second
        min_silence_len=20,

        # consider it silent if quieter than -16 dBFS
        silence_thresh=-(round(abs(sound.dBFS))+100)
    )

    #SPEAKING RATE
    speaking_rate=len(hunks)/len(chunks)
    save.append(speaking_rate)
    
    #SILENCES
    pauses=detect_silence(sound, min_silence_len=20, silence_thresh=-(round(abs(sound.dBFS))+100), seek_step=1)
    try:
        save.append(pauses[0][1])
    except:
          save.append(0)
    #FINDING DURATIONS
    sumd=0
    ls=[]
    for start_i, end_i in pauses:
            dur=int(end_i)-int(start_i)
            ls.append(dur)
    breaks=[]

    #VOICE BREAKS
    for i in ls:
        if i >900:
            breaks.append(i)
    save.append(len(breaks))
    #SEPRATING BREAKES AND PAUSES
    ls=list(set(ls)-set(breaks))
    #PUASES DURATION
    for i in ls:
        sumd=sumd+i
    save.append(sumd)
    #NO OF PAUSES
    nop=len(ls)
    save.append(nop)
    #NO OF VOICE BREAKES
    noofVoiceBreaks=len(breaks)
    save.append(noofVoiceBreaks)
    #MAXIMUM DURATION OF PAUSES
    try:
        maximum_pause=max(ls)
    except:
        maximum_pause=0
    save.append(maximum_pause)
    #AVG DURATION OF PAUSES
    try:
        avgp=sumd/nop
    except:
        avgp =0
    save.append(avgp)
    #FINDING PEAK
    peak=scipy.signal.find_peaks(data,rel_height=0.5)
    #MAXIMUM FALLING AND MAXIMUM RISING
    MaxFalling=np.amin(data)
    save.append(MaxFalling)
    MaxRising=np.amax(peak[0])
    save.append(MaxRising)

    #JITTER, SHIMMER, JITTERRAP 
    sums=0
    for i in range(1,len(peak[0])-1):
        sums=sums+abs(20*math.log10(peak[0][i+1]/peak[0][i]))
    #SHIMMER
    shimmer=sums/(len(peak[0])-1)
    peakf=abs(np.fft.fft(peak[0]))
    sumps=0
    for i in range(1,len(peakf)-1):
        sumps=sumps+(peakf[i+1]**-1)-(peakf[i]**-1)

    #JITTER
    jitter=sumps/(len(peakf)-1)
    sortedp=np.sort(peak[0])
    sortedf=abs(np.fft.fft(sortedp))
    dif=abs(sortedp[11]-sortedp[15])
    suh=0
    avgabsdiff=(dif)/4
    avgneigh1=(abs(sortedp[6]-sortedp[10]))
    avgneigh2=abs(sortedp[17]-sortedp[22])
    avg=(dif+avgneigh1+avgneigh2)/3

    for i in range(11,16):
        suh=suh+abs(sortedf[i]**-1)
    period=suh/5

    #JITTERRAP
    jitterrap=(avgabsdiff+avg)/period

    save.append(jitter)
    save.append(shimmer)
    save.append(jitterrap)
    
    #NUMBER OF RISING, NUMBER OF FALLING, AVERAGE RISE, AVERAGE FALL

    noofrise=len(peak[0])
    
    avgtorise=noofrise/len(data)
    nooffall=0
    for i in data:   
        if i == np.amin(data):
            nooffall=nooffall+1

    avgtofall=nooffall/len(data)
    save.append(noofrise)
    save.append(nooffall)
    save.append(avgtorise)
    save.append(avgtofall)

    # ZCR
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    save.append(zcr[0]) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=16000).T, axis=0)
    save.append(np.mean(chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=16000).T, axis=0)
    save.append(np.mean(mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    save.append(rms[0]) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=16000).T, axis=0)
    save.append(np.mean(mel)) # stacking horizontally

    # Spectral Bandwidth
    sb = np.mean(librosa.feature.spectral_bandwidth(y=data,sr=16000), axis=0)
    save.append(np.mean(mel))

    # Spectral Centroid
    sc = np.mean(librosa.feature.spectral_centroid(y=data,sr=16000), axis=0)
    save.append(np.mean(mel))
    df.loc[len(df)] = save
    save = []
    print(index)
    

In [ ]:
df.to_csv("statistic data.csv")

In [ ]:
#Loading data

save = []
for index,file in enumerate(os.listdir(files_dir)[5000:5500]):
    save.append(file[:-4])
    save.append(label_df["professional-diagnosis"].loc[label_df["file_name"]==file].values[0])
    data, sampling_rate  = librosa.core.load(os.path.join(files_dir,file))
    Audio(data = data,rate = sampling_rate)
    librosa.display.waveshow(data, sampling_rate)
    # Duration : Total duration of the audio
    total_samples = data.size
    total_duration = int(total_samples/sampling_rate)
    save.append(total_duration)
    # Fourier Transform from data 

    fourier_transform = np.fft.rfft(data)

    # Frequencies

    frequencies = np.abs(fourier_transform)
    
    # Maximum Pitch

    max_pitch = np.argmax(frequencies)
    save.append(max_pitch)

    arr = [117]*len(frequencies)
    sum_freq = 0
    for i in range(0,len(frequencies)):
        if int(frequencies[i])>0 and int(frequencies[i])<int(max_pitch):
            arr[i] = int(frequencies[i])
            sum_freq=sum_freq + i
    # Minmum Pitch
    min_pitch = arr.index(min(arr))
    save.append(min_pitch)
    # Mean pitch
    mean_pitch=sum_freq/len(frequencies)
    save.append(mean_pitch)
    #total energy
    energy= np.sum(data**2)
    save.append(energy)
    energy_freq = np.sum(frequencies**2)/total_samples
    save.append(energy_freq)
    #power
    power = energy / total_duration
    save.append(power)
    #minimum intensity
    plt.figure(1, figsize=(9,6))
    plt.subplot(211)
    #IntensityMin : Minimum Intensity of the audio signal
    Pxx, freqs, bins, im = plt.specgram(data, Fs = sampling_rate, NFFT=1024, cmap=plt.get_cmap('autumn_r'))
    col_bar=plt.colorbar(im)
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    col_bar.set_label('Intensity dB')
    plt.show()
    plt.close()
    min_intensity = col_bar.vmin
    save.append(min_intensity)
    #maximum intensity
    max_intensity = col_bar.vmax
    save.append(max_intensity)
    #mean intensity
    mean_intensity = (min_intensity + max_intensity) / 2
    save.append(mean_intensity)

    sound = AudioSegment.from_wav(os.path.join(files_dir,file))
    len(sound)
    chunks = split_on_silence(sound, 
    # must be silent for at least half a second
    min_silence_len=20,

    # consider it silent if quieter than -16 dBFS
        silence_thresh=-(round(abs(sound.dBFS))+100)
    )



    # 1000 means 1 sec then 60 second means
    sixty_seconds = 60 * 1000

    second_1_minute = sound[sixty_seconds:120000]
    hunks = split_on_silence(second_1_minute, 
        # must be silent for at least half a second
        min_silence_len=20,

        # consider it silent if quieter than -16 dBFS
        silence_thresh=-(round(abs(sound.dBFS))+100)
    )

    #SPEAKING RATE
    speaking_rate=len(hunks)/len(chunks)
    save.append(speaking_rate)
    
    #SILENCES
    pauses=detect_silence(sound, min_silence_len=20, silence_thresh=-(round(abs(sound.dBFS))+100), seek_step=1)
    try:
        save.append(pauses[0][1])
    except:
          save.append(0)
    #FINDING DURATIONS
    sumd=0
    ls=[]
    for start_i, end_i in pauses:
            dur=int(end_i)-int(start_i)
            ls.append(dur)
    breaks=[]

    #VOICE BREAKS
    for i in ls:
        if i >900:
            breaks.append(i)
    save.append(len(breaks))
    #SEPRATING BREAKES AND PAUSES
    ls=list(set(ls)-set(breaks))
    #PUASES DURATION
    for i in ls:
        sumd=sumd+i
    save.append(sumd)
    #NO OF PAUSES
    nop=len(ls)
    save.append(nop)
    #NO OF VOICE BREAKES
    noofVoiceBreaks=len(breaks)
    save.append(noofVoiceBreaks)
    #MAXIMUM DURATION OF PAUSES
    try:
        maximum_pause=max(ls)
    except:
        maximum_pause=0
    save.append(maximum_pause)
    #AVG DURATION OF PAUSES
    try:
        avgp=sumd/nop
    except:
        avgp =0
    save.append(avgp)
    #FINDING PEAK
    peak=scipy.signal.find_peaks(data,rel_height=0.5)
    #MAXIMUM FALLING AND MAXIMUM RISING
    MaxFalling=np.amin(data)
    save.append(MaxFalling)
    MaxRising=np.amax(peak[0])
    save.append(MaxRising)

    #JITTER, SHIMMER, JITTERRAP 
    sums=0
    for i in range(1,len(peak[0])-1):
        sums=sums+abs(20*math.log10(peak[0][i+1]/peak[0][i]))
    #SHIMMER
    shimmer=sums/(len(peak[0])-1)
    peakf=abs(np.fft.fft(peak[0]))
    sumps=0
    for i in range(1,len(peakf)-1):
        sumps=sumps+(peakf[i+1]**-1)-(peakf[i]**-1)

    #JITTER
    jitter=sumps/(len(peakf)-1)
    sortedp=np.sort(peak[0])
    sortedf=abs(np.fft.fft(sortedp))
    dif=abs(sortedp[11]-sortedp[15])
    suh=0
    avgabsdiff=(dif)/4
    avgneigh1=(abs(sortedp[6]-sortedp[10]))
    avgneigh2=abs(sortedp[17]-sortedp[22])
    avg=(dif+avgneigh1+avgneigh2)/3

    for i in range(11,16):
        suh=suh+abs(sortedf[i]**-1)
    period=suh/5

    #JITTERRAP
    jitterrap=(avgabsdiff+avg)/period

    save.append(jitter)
    save.append(shimmer)
    save.append(jitterrap)
    
    #NUMBER OF RISING, NUMBER OF FALLING, AVERAGE RISE, AVERAGE FALL

    noofrise=len(peak[0])
    
    avgtorise=noofrise/len(data)
    nooffall=0
    for i in data:   
        if i == np.amin(data):
            nooffall=nooffall+1

    avgtofall=nooffall/len(data)
    save.append(noofrise)
    save.append(nooffall)
    save.append(avgtorise)
    save.append(avgtofall)

    # ZCR
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    save.append(zcr[0]) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=16000).T, axis=0)
    save.append(np.mean(chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=16000).T, axis=0)
    save.append(np.mean(mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    save.append(rms[0]) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=16000).T, axis=0)
    save.append(np.mean(mel)) # stacking horizontally

    # Spectral Bandwidth
    sb = np.mean(librosa.feature.spectral_bandwidth(y=data,sr=16000), axis=0)
    save.append(np.mean(mel))

    # Spectral Centroid
    sc = np.mean(librosa.feature.spectral_centroid(y=data,sr=16000), axis=0)
    save.append(np.mean(mel))
    df.loc[len(df)] = save
    save = []
    print(index)
    

In [ ]:
df.to_csv("statistic_test_data.csv")